In [ ]:
# 下記セルを実行すると、Googleドライブへの接続を求められます。
# Googleアカウントにログインして進めてください。
import os
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
working_dir = 'MLSys_100Knocks' #　※※自分で作成したフォルダパスが異なる場合こちらを変更してください。※※
path = f'/content/drive/MyDrive/{working_dir}/本章/2章'
os.chdir(path)
! pip install japanize-matplotlib

# ２章 データを可視化し分析を行う１０本ノック



### ノック１１：データを読み込んで不要なものを除外しよう

In [ ]:
import pandas as pd
order_data = pd.read_csv('order_data.csv')
print(len(order_data))
order_data.head()

In [ ]:
order_data = order_data.loc[(order_data['status'] == 1)|(order_data['status'] == 2)]
print(len(order_data))
order_data.columns

In [ ]:
analyze_data = order_data[['store_id', 'customer_id', 'coupon_cd',
       'order_accept_date', 'delivered_date', 'total_amount',
       'store_name', 'wide_area', 'narrow_area',
       'takeout_name', 'status_name']]
print(analyze_data.shape)
analyze_data.head()

### ノック１２：データの全体像を把握しよう

In [ ]:
analyze_data.describe()

In [ ]:
analyze_data.dtypes

In [ ]:
analyze_data[['store_id', 'coupon_cd']] = analyze_data[['store_id', 'coupon_cd']].astype(str)
analyze_data.dtypes

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
analyze_data.describe()

### ノック１３：月別の売上を集計してみよう

In [ ]:
analyze_data['order_accept_date'] = pd.to_datetime(analyze_data['order_accept_date'])
analyze_data['order_accept_month'] = analyze_data['order_accept_date'].dt.strftime('%Y%m')
analyze_data[['order_accept_date', 'order_accept_month']].head()

In [ ]:
analyze_data['delivered_date'] = pd.to_datetime(analyze_data['delivered_date'])
analyze_data['delivered_month'] = analyze_data['delivered_date'].dt.strftime('%Y%m')
analyze_data[['delivered_date', 'delivered_month']].head()

In [ ]:
analyze_data.dtypes

In [ ]:
month_data = analyze_data.groupby('order_accept_month')
month_data.describe()

In [ ]:
month_data['total_amount'].sum()

### ノック１４：月別の推移を可視化してみよう

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
month_data['total_amount'].sum().plot()

In [ ]:
month_data['total_amount'].mean().plot()

### ノック１５：売上からヒストグラムを作成してみよう

In [ ]:
plt.hist(analyze_data['total_amount'])

In [ ]:
plt.hist(analyze_data['total_amount'], bins=21)

### ノック１６：都道府県別の売上を集計して可視化しよう

In [ ]:
pre_data = pd.pivot_table(analyze_data, index='order_accept_month', columns='narrow_area', values='total_amount', aggfunc='mean')
pre_data

In [ ]:
import japanize_matplotlib
plt.plot(list(pre_data.index), pre_data['東京'], label='東京')
plt.plot(list(pre_data.index), pre_data['神奈川'], label='神奈川')
plt.plot(list(pre_data.index), pre_data['埼玉'], label='埼玉')
plt.plot(list(pre_data.index), pre_data['千葉'], label='千葉')
plt.plot(list(pre_data.index), pre_data['茨城'], label='茨城')
plt.plot(list(pre_data.index), pre_data['栃木'], label='栃木')
plt.plot(list(pre_data.index), pre_data['群馬'], label='群馬')
plt.legend()

### ノック１７：クラスタリングに向けてデータを加工しよう

In [ ]:
store_clustering = analyze_data[['store_id', 'total_amount']].groupby('store_id').agg(['size', 'mean', 'median', 'max', 'min'])
store_clustering.reset_index(inplace = True, drop = True)
print(len(store_clustering))
store_clustering.head()

In [ ]:
import seaborn as sns
hexbin = sns.jointplot(x='mean', y='size', data=store_clustering['total_amount'], kind='hex')

### ノック１８：クラスタリングで店舗をグループ化してみよう

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
store_clustering_sc = sc.fit_transform(store_clustering)

kmeans = KMeans(n_clusters=4, random_state=0)
clusters = kmeans.fit(store_clustering_sc)
store_clustering['cluster'] = clusters.labels_
print(store_clustering['cluster'].unique())
store_clustering.head()

### ノック１９：グループの傾向を分析してみよう

In [ ]:
store_clustering.columns = ['月内件数', '月内平均値', '月内中央値', '月内最大値', '月内最小値', 'cluster']
store_clustering.groupby('cluster').count()

In [ ]:
store_clustering.groupby('cluster').mean()

### ノック２０：クラスタリングの結果をt-SNEで可視化しよう

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
x = tsne.fit_transform(store_clustering_sc)
tsne_df = pd.DataFrame(x)
tsne_df['cluster'] = store_clustering['cluster']
tsne_df.columns = ['axis_0', 'axis_1', 'cluster']
tsne_df.head()

In [ ]:
tsne_graph = sns.scatterplot(x='axis_0', y='axis_1', hue='cluster', data=tsne_df)